In [5]:
import os
import time
import json
from datetime import datetime, timezone

import requests
import pandas as pd

BASE_URL = "https://api.llama.fi"

RAW_DIR = os.path.join("..", "data", "raw")
os.makedirs(RAW_DIR, exist_ok=True)

RUN_TS = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S")
print("Run timestamp UTC:", RUN_TS)
print("Raw dir:", os.path.abspath(RAW_DIR))


Run timestamp UTC: 20251227_190951
Raw dir: c:\Users\David\defi_protocol_quality\data\raw


In [6]:
def get_json(url: str, params: dict | None = None, retries: int = 3, sleep_s: float = 1.0) -> dict:
    last_err = None
    for attempt in range(1, retries + 1):
        try:
            r = requests.get(url, params=params, timeout=30)
            r.raise_for_status()
            return r.json()
        except Exception as e:
            last_err = e
            print(f"Request failed attempt {attempt} of {retries}")
            print("URL:", url)
            time.sleep(sleep_s * attempt)
    raise RuntimeError(f"Failed after {retries} retries. Last error: {last_err}")


In [7]:
protocols = get_json(f"{BASE_URL}/protocols")
df_protocols = pd.DataFrame(protocols)

print("Protocols count:", len(df_protocols))
df_protocols[["name", "slug", "tvl"]].head(10)


Protocols count: 6865


,name,slug,tvl
0,Binance CEX,binance-cex,1.637486e+11
1,Aave V3,aave-v3,3.248886e+10
2,Lido,lido,2.569153e+10
3,OKX,okx,2.230732e+10
4,Bitfinex,bitfinex,2.004217e+10
5,Bybit,bybit,1.848204e+10
6,Robinhood,robinhood,1.621312e+10
7,EigenCloud,eigencloud,1.210293e+10
8,WBTC,wbtc,1.084052e+10
9,Binance staked ETH,binance-staked-eth,1.037416e+10


In [15]:
import sys
!{sys.executable} -m pip install pyarrow



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
# Save full raw as JSON for safety (always works)
protocols_json_path = RAW_DIR + "/protocols_" + RUN_TS + ".json"
df_protocols.to_json(protocols_json_path, orient="records")
print("Saved raw JSON to:", protocols_json_path)

# Save a clean subset to Parquet (stable schema)
keep_cols = ["name", "slug", "tvl", "chain", "category"]
clean_cols = [c for c in keep_cols if c in df_protocols.columns]

df_protocols_clean = df_protocols[clean_cols].copy()

protocols_parquet_path = RAW_DIR + "/protocols_clean_" + RUN_TS + ".parquet"
df_protocols_clean.to_parquet(protocols_parquet_path, index=False)
print("Saved clean Parquet to:", protocols_parquet_path)

df_protocols_clean.head(3)


Saved raw JSON to: ..\data\raw/protocols_20251227_191015.json
Saved clean Parquet to: ..\data\raw/protocols_clean_20251227_191015.parquet


,name,slug,tvl,chain,category
0,Binance CEX,binance-cex,1.637486e+11,Multi-Chain,CEX
1,Aave V3,aave-v3,3.248886e+10,Multi-Chain,Lending
2,Lido,lido,2.569153e+10,Multi-Chain,Liquid Staking


In [12]:
TARGET_NAMES = ["Curve", "Lido", "GMX", "OlympusDAO", "SushiSwap"]

d = df_protocols.copy()
d["name_lower"] = d["name"].astype(str).str.lower()

resolved = []
for target in TARGET_NAMES:
    t = target.lower().replace("dao", "").strip()

    # prefer exact name match first
    exact = d[d["name_lower"] == target.lower()]
    if len(exact) > 0:
        row = exact.sort_values("tvl", ascending=False).iloc[0]
    else:
        # fallback to contains match
        contains = d[d["name_lower"].str.contains(t, na=False)]
        row = contains.sort_values("tvl", ascending=False).iloc[0]

    resolved.append(
        {
            "target": target,
            "matched_name": row["name"],
            "slug": row["slug"],
            "tvl": row.get("tvl"),
            "chain": row.get("chain"),
            "category": row.get("category"),
        }
    )

df_resolved = pd.DataFrame(resolved)
df_resolved


,target,matched_name,slug,tvl,chain,category
0,Curve,Curve DEX,curve-dex,2.135340e+09,Multi-Chain,Dexs
1,Lido,Lido,lido,2.569153e+10,Multi-Chain,Liquid Staking
2,GMX,GMX V2 Perps,gmx-v2-perps,3.806784e+08,Multi-Chain,Derivatives
3,OlympusDAO,Olympus DAO,olympus-dao,0.000000e+00,Ethereum,Reserve Currency
4,SushiSwap,SushiSwap,sushiswap,1.025940e+08,Multi-Chain,Dexs


In [13]:
case_map_path = RAW_DIR + "/case_protocols_" + RUN_TS + ".csv"
df_resolved.to_csv(case_map_path, index=False)
print("Saved case mapping to:", case_map_path)


Saved case mapping to: ..\data\raw/case_protocols_20251227_191015.csv


In [14]:
def fetch_protocol_bundle(slug: str) -> dict:
    return get_json(f"{BASE_URL}/protocol/{slug}")

bundles = {}
for slug in df_resolved["slug"]:
    print("Fetching:", slug)
    bundles[slug] = fetch_protocol_bundle(slug)
    time.sleep(0.4)

print("Fetched bundles:", len(bundles))


Fetching: curve-dex
Fetching: lido
Fetching: gmx-v2-perps
Fetching: olympus-dao
Fetching: sushiswap
Fetched bundles: 5


In [17]:
bundles_path = RAW_DIR + "/protocol_bundles_" + RUN_TS + ".json"
with open(bundles_path, "w", encoding="utf-8") as f:
    json.dump(bundles, f)
print("Saved bundles to:", bundles_path)


Saved bundles to: ..\data\raw/protocol_bundles_20251227_191015.json


In [18]:
slug_to_name = dict(zip(df_resolved["slug"], df_resolved["matched_name"]))

rows = []
for slug, data in bundles.items():
    name = slug_to_name.get(slug, slug)
    series = data.get("tvl", []) or []
    for p in series:
        ts = int(p["date"])
        rows.append(
            {
                "date": pd.to_datetime(ts, unit="s", utc=True).date(),
                "protocol": name,
                "slug": slug,
                "tvl_usd": float(p["totalLiquidityUSD"]),
            }
        )

df_tvl = pd.DataFrame(rows).sort_values(["protocol", "date"]).reset_index(drop=True)
df_tvl.head()


,date,protocol,slug,tvl_usd
0,2020-02-09,Curve DEX,curve-dex,1163733.0
1,2020-02-10,Curve DEX,curve-dex,1372109.0
2,2020-02-11,Curve DEX,curve-dex,8936.0
3,2020-02-12,Curve DEX,curve-dex,38139.0
4,2020-02-13,Curve DEX,curve-dex,273331.0


In [19]:
tvl_path = RAW_DIR + "/protocol_tvl_daily_" + RUN_TS + ".parquet"
df_tvl.to_parquet(tvl_path, index=False)
print("Saved protocol TVL daily to:", tvl_path)
print("Rows:", len(df_tvl), "Protocols:", df_tvl["protocol"].nunique())


Saved protocol TVL daily to: ..\data\raw/protocol_tvl_daily_20251227_191015.parquet
Rows: 8497 Protocols: 5


In [20]:
chain_rows = []
for slug, data in bundles.items():
    name = slug_to_name.get(slug, slug)
    chain_tvls = data.get("chainTvls", {}) or {}

    if not isinstance(chain_tvls, dict) or len(chain_tvls) == 0:
        print("No chain breakdown for:", slug)
        continue

    for chain, chain_obj in chain_tvls.items():
        series = (chain_obj or {}).get("tvl", []) or []
        for p in series:
            ts = int(p["date"])
            chain_rows.append(
                {
                    "date": pd.to_datetime(ts, unit="s", utc=True).date(),
                    "protocol": name,
                    "slug": slug,
                    "chain": str(chain),
                    "tvl_usd": float(p["totalLiquidityUSD"]),
                }
            )

df_chain = pd.DataFrame(chain_rows)
if len(df_chain) > 0:
    df_chain = df_chain.sort_values(["protocol", "chain", "date"]).reset_index(drop=True)

df_chain.head()


,date,protocol,slug,chain,tvl_usd
0,2021-09-13,Curve DEX,curve-dex,Arbitrum,1274240.0
1,2021-09-14,Curve DEX,curve-dex,Arbitrum,9574829.0
2,2021-09-15,Curve DEX,curve-dex,Arbitrum,52104273.0
3,2021-09-16,Curve DEX,curve-dex,Arbitrum,134762216.0
4,2021-09-17,Curve DEX,curve-dex,Arbitrum,128576159.0


In [21]:
chain_path = RAW_DIR + "/protocol_chain_tvl_daily_" + RUN_TS + ".parquet"
df_chain.to_parquet(chain_path, index=False)
print("Saved chain TVL daily to:", chain_path)
print("Rows:", len(df_chain))


Saved chain TVL daily to: ..\data\raw/protocol_chain_tvl_daily_20251227_191015.parquet
Rows: 83699
